In [2]:
import sys
import warnings
warnings.filterwarnings('ignore')
!{sys.executable} -m pip install pandapower

## Initial AC power flow Model for IEEE 30 System

In [3]:
import pandapower as pp
import pandas as pd
import random
import pandapower.networks as pn
import numpy as np
from sklearn.utils import shuffle
import torch
net = pn.case_ieee30()
pp.runpp(net,numba=False) # Solve initial power flow
V = net.res_bus.vm_pu  # Voltage magnitudes
theta = net.res_bus.va_degree  # Voltage angles
num_buses = len(net.bus)
dataset = pd.DataFrame(columns=[f"V{i}" for i in range(1, num_buses + 1)] +
                          [f"theta{i}" for i in range(1, num_buses + 1)] +
                          [ "label"])




## Steps to Generate dataset


1.   Simulating Line deletions
2.   Changing the load of network
3.   Solve power flow for modified system
4.   Extract features for modified system
5.   Calculate deviations from initial state
6.   Create feature vector and label
7.   Add noise to voltage and angle measurements
8.   Append to dataset
9.   Restore lines and load  for next iteration
10.  At end of iteration, export the dataset to csv file.







In [ ]:
while len(dataset) < 35000:

    for line_id in net.line.index:
        if(len(dataset) > 1000):
            net.line.loc[line_id, "in_service"] = False
        load_factor = random.uniform(0.8, 1.2)
        net.load.scaling = load_factor
        pp.runpp(net,numba=False)

        V_prime = net.res_bus.vm_pu
        theta_prime = net.res_bus.va_degree

        dV = V_prime - V
        dtheta = theta_prime - theta

        data_point = pd.Series({**{f"V{i}": dV[i - 1] for i in range(1, num_buses + 1)},
                                  **{f"theta{i}": dtheta[i - 1] for i in range(1, num_buses + 1)},
                                  **{"label": line_id}})  

        dV += np.random.normal(scale=0.01, size=dV.shape)
        dtheta += np.random.normal(scale=0.01, size=dtheta.shape)

        dataset = dataset.append(data_point, ignore_index=True)
        
        if(len(dataset) > 1000):
            net.line.loc[line_id, "in_service"] = True
        net.load.scaling /= load_factor


dataset=shuffle(dataset)
dataset.to_csv("single_line.csv", index=False)

In [418]:
net

This pandapower network includes the following parameter tables:
   - bus (30 element)
   - load (21 element)
   - gen (5 element)
   - shunt (2 element)
   - ext_grid (1 elements)
   - line (34 element)
   - trafo (7 element)
   - poly_cost (6 element)
   - bus_geodata (30 element)
 and the following results tables:
   - res_bus (30 element)
   - res_line (34 element)
   - res_trafo (7 element)
   - res_ext_grid (1 elements)
   - res_load (21 element)
   - res_shunt (2 element)
   - res_gen (5 element)

In [ ]:
df = pd.read_csv("single_line.csv")
df.head()

## Modelling

In [372]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split  # For splitting into training and validation sets
data = pd.read_csv("single_line.csv")
X_train, X_test, y_train, y_test = train_test_split(data.drop("label", axis=1), data["label"], test_size=0.2)



In [387]:
print('Features rows:', X_train.shape[0])
print('Labels rows:', y_train.shape[0])


Features rows: 28016
Labels rows: 28016


In [386]:
X_train.fillna(0,inplace=True)
y_train.fillna(0,inplace=True)

In [ ]:
X_train_array = X_train.values
X_train_tensor = torch.from_numpy(X_train_array)
X_train_tensor = X_train_tensor.float().to("cuda" if torch.cuda.is_available() else "cpu")
y_train_array = y_train.values
y_train_tensor = torch.from_numpy(y_train)
y_train_tensor = y_train_tensor.float().to("cuda" if torch.cuda.is_available() else "cpu")

In [502]:

X_test_array = X_test.values
X_test_tensor = torch.from_numpy(X_test_array)
X_test_tensor = X_test_tensor.float().to("cuda" if torch.cuda.is_available() else "cpu")
y_test_array = y_test.values
y_test_tensor = torch.from_numpy(y_test_array)
y_test_tensor = y_test_tensor.float().to("cuda" if torch.cuda.is_available() else "cpu")

In [437]:
X_train.shape

torch.Size([28016, 60])

In [459]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
torch.manual_seed(42)

In [460]:
n_inputs = X_train.shape[1]
num_classes = 34


In [523]:
class MultinomialLogisticRegression(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MultinomialLogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        logits = self.linear(x)
        return F.log_softmax(logits, dim=1)  

model = MultinomialLogisticRegression(n_inputs,num_classes).to("cuda" if torch.cuda.is_available() else "cpu")
output = model(X_train_tensor)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0002)

for epoch in range(10):
    running_loss = 0.0


    for i in range(len(X_train_tensor)):
        inputs = X_train_tensor[i]
        labels = y_train_tensor[i]
        optimizer.zero_grad()
        outputs = model(inputs.unsqueeze(0))
        loss = criterion(outputs, labels.unsqueeze(0).long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

In [531]:
correct = 0
total = 0

with torch.no_grad():
    for i in range(len(X_test_tensor)):
        inputs = X_test_tensor[i]
        labels = y_test_tensor[i]
        outputs = model(inputs.unsqueeze(0))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.unsqueeze(0).size(0)
        correct += (predicted == labels).sum().item()
        accuracy = 100 * correct // total


print(f'Accuracy of the network on the  test dataset : {accuracy} %')

Accuracy of the network on the  test dataset : 99 %


In [567]:
y_train_tensor.shape[0]

28016

In [575]:
sample_index = np.random.randint(1,y_test_tensor.shape[0])
inputs = X_test_tensor[sample_index]
output = model(inputs.unsqueeze(0))
predicted_outage_index = output.argmax()

line_labels = ["Line 1", "Line 2", "Line 3","Line 4","Line 5",
               "Line 6","Line 7", "Line 8","Line 9", "Line 10",
               "Line 11","Line 12", "Line 13","Line 14", "Line 15",
               "Line 16","Line 17", "Line 18","Line 19", "Line 20",
               "Line 21","Line 22", "Line 23","Line 24", "Line 25",
               "Line 26","Line 27", "Line 28","Line 29", "Line 30",
               "Line 31","Line 32", "Line 33","Line 34"]

predicted_outage_line = line_labels[predicted_outage_index]

actual_outage_index = int(y_test_tensor[sample_index].item())
actual_outage_line = line_labels[actual_outage_index]


print(f"Predicted outage line: {predicted_outage_line}")
print(f"Actual outage line:     {actual_outage_line}")

Predicted outage line: Line 17
Actual outage line:     Line 17
